**Exercise 1**
<br/>
Implement the final version of the authenticated DH protocol in Slide 16. (You can
use external libraries.)

In [3]:
!pip install rsa

In [32]:
from sympy import randprime, primitive_root, factorint, isprime
import rsa
import random

class DH_Participant:

    __name: str
    __private_key: int
    __public_key: int
    __shared_private_key: int
    __p: int


    def __init__(self, name: str):
        self.__name = name


    # private function
    def __generate_q(self, p: int):
        factors = factorint(p-1)
        prime_factors = [factor for factor in factors if isprime(factor)]

        return max(prime_factors) if prime_factors else None


    # private function
    def __generate_r(self, p: int, g: int):
        while True:
            r = random.randint(2, p-2)
            if pow(r, 2, p) != 1 and pow(g, r, p) != 1 and pow(g, r, p) != p - 1:
                return r


    # private function
    def __generate_authentication(self, msg: str):
        rsa_public_key, rsa_private_key = rsa.newkeys(2048)
        signature = rsa.sign(msg.encode('utf-8'), rsa_private_key, 'SHA-256')
        return (rsa_public_key, signature)


    # private function
    def __verify_authentication(self, msg: str, authentications):
        (rsa_public_key, signature) = authentications

        return rsa.verify(msg.encode('utf-8'), signature, rsa_public_key)


    # generate s as the minimum size of p
    # means that if s= 4, p need at least 4 bits to present
    def generate_s(self):
        # limit min between 2**8 to 2**16
        s = random.randint(8,16)
        print(self.__name, f"chose s as {s}")
        return s


    # generate parameters based on s
    # return neccessary params, public key and authentication info using RSA algorithm
    def select_parameters(self, s: int):
        p = randprime(2**(s-1), 2**s)
        self.__p = p
        g = primitive_root(p)
        q = self.__generate_q(p)
        r = self.__generate_r(p, g)
        print(f"{self.__name} chose p as {p}, q as {q}, r as {r} with generator as {g}")
        self_private_key = random.randint(1, q - 1)
        self.__private_key = self_private_key
        self_public_key = pow(g, self_private_key, p)
        self.__public_key = self_public_key
        print(f"{self.__name}'s private key is {self.__private_key}")
        print(f"{self.__name}'s public key is {self.__public_key}")
        message_authen = f"{g},{p},{q},{r},{self_public_key}"

        auth = self.__generate_authentication(message_authen)
        return (g, p, q, r), self_public_key, auth


    # receive params from partner, calculate public, private and shared secret key
    # return public key, authentication info using RSA algorithm
    def receive_initial_parameters(self, params):
        (g, p, q, r), public_key, auth = params
        self.__p = p

        message_authen = f"{g},{p},{q},{r},{public_key}"
        self.__verify_authentication(message_authen, auth)
        print(f"{self.__name} received successfully")

        self_private_key = random.randint(1, q - 1)
        self.__private_key = self_private_key
        self_public_key = pow(g, self_private_key, p)
        self.__public_key = self_public_key
        print(f"{self.__name}'s private key is {self.__private_key}")
        print(f"{self.__name}'s public key is {self.__public_key}")

        shared_secret = pow(public_key, self_private_key, p)
        self.__shared_private_key = shared_secret
        print(f"{self.__name}'s shared secret key is {self.__shared_private_key}")

        auth = self.__generate_authentication(f"{self_public_key}")
        return self_public_key, auth


    # receive authentication params from partner, calculate shared secret key
    def receive_partner_public_key(self, partner_public_key, auth_params):
        self.__verify_authentication(f"{partner_public_key}", auth_params)
        print(f"{self.__name} received successfully")
        shared_secret = pow(partner_public_key, self.__private_key, self.__p)
        self.__shared_private_key = shared_secret
        print(f"{self.__name}'s shared secret key is {self.__shared_private_key}")


# initial 2 object Alice and Bob
alice = DH_Participant("Alice")
bob = DH_Participant("Bob")
# step 1: Alice select s as minimum size of the prime
s = alice.generate_s()
# step 2: Bob generate params based on s
params = bob.select_parameters(s)

try:
    # step 3: Alice receive the params from Bob, verify then calculate the shared secret for itself
    alice_public_key,auth_params = alice.receive_initial_parameters(params)
    # step 4: Bob receive the authentication params from Alice, verify then calculate the shared secret itself
    bob.receive_partner_public_key(alice_public_key, auth_params)
except:
    print("invalid signature!")


Alice chose s as 10
Bob chose p as 743, q as 53, r as 731 with generator as 5
Bob's private key is 47
Bob's public key is 434
Alice received successfully
Alice's private key is 36
Alice's public key is 333
Alice's shared secret key is 659
Bob received successfully
Bob's shared secret key is 659


**Exercise 4**
<br/>
Trusted Third Party (TTP) might be involved in a fair non-repudiation protocol in different extents
-  **In-line TTP** acts as an intermediary between the originator and the recipient
and intervenes directly in a non-repudiation service (e.g. the protocol in Slide
36).
- **On-line TTP** is actively involved in every instance of a non-repudiation service (e.g. the protocol in Slide 38).
- **Off-line TTP** supports non-repudiation without being involved in each instance of a service

Design (or find in the literature, e.g., in IEEE CSF 1997, “An Efficient Non repudiation Protocol”) a fair non-repudiation protocol using an **off-line TTP** which
does not need to be involved unless the originator or the recipient misbehaves.